In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("train.csv")

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
df=df[["Order Date","Product ID","Sales"]]

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:

DATE_COL = "Order Date"
ID_COL="Product ID"
TARGET_COL = "Sales"
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
FORECAST_HORIZON=18



In [ ]:
from datetime import date, timedelta
monthly_aggrigated_ecommerce_data=pd.DataFrame()
        
        
for article in df[ID_COL].unique():
    sub_df = df[(df[ID_COL]==article)]
    end_date_last_day = sub_df[DATE_COL].max().replace(day=1) - timedelta(days=1)
    

    #end_date_last_day = date.today().replace(day=1) - timedelta(days=1)
    
    end_date_last_day=pd.to_datetime(end_date_last_day)


#filter out the transactions or drop current month demand as it is progressively get updated
    sub_df = sub_df[(sub_df[DATE_COL] <= end_date_last_day)]
    if not(sub_df.empty):
        sub_df_copy = sub_df.copy()
        #add a record to fill 0s for discontinued product untill last month
        if(sub_df[DATE_COL].max() < end_date_last_day):
            record_to_insert = {ID_COL: article, DATE_COL: end_date_last_day, TARGET_COL : 0} #populate a dummy record of last month so that resampling function handles filling 0s for discontinued products 
            sub_df = sub_df.append(record_to_insert, ignore_index=True)

        #sub_df = sub_df.append(generate_forecast_window(end_date_last_day, FORECAST_HORIZON, article, customer, cost=sub_df["COST"].iloc[-1]), ignore_index=True)
        #df['date'] = pd.to_datetime(df['date'])
        sub_df[DATE_COL]=pd.to_datetime(sub_df[DATE_COL])
        sub_df.index = sub_df[DATE_COL]
        sub_df = sub_df.resample(rule='MS').agg({'Sales' : 'sum'})
        sub_df = sub_df.reset_index()
        sub_df[ID_COL] = article  
        monthly_aggrigated_ecommerce_data = monthly_aggrigated_ecommerce_data.append(sub_df, ignore_index=True)

monthly_aggrigated_ecommerce_data.to_csv("output.csv",index = False)


In [ ]:
monthly_aggrigated_ecommerce_data[ID_COL].unique()

In [ ]:
#monthly_aggrigated_ecommerce_data =monthly_aggrigated_ecommerce_data[monthly_aggrigated_ecommerce_data[ID_COL] == "FUR-BO-10001798"]

In [ ]:

def split_time_series_for_modeling(df, evaluation_size):
            import pandas as pd 
            import numpy as np
            train_size = len(df) - evaluation_size
            train, eval = df.iloc[:train_size], df.iloc[train_size:]
            return train, eval
train_df1=pd.DataFrame()
eval_df1=pd.DataFrame()
        
for article in monthly_aggrigated_ecommerce_data[ID_COL].unique():
    single_article_df = monthly_aggrigated_ecommerce_data[monthly_aggrigated_ecommerce_data[ID_COL] == article]
    if not(single_article_df.empty):
        train_df, evaluation_df = split_time_series_for_modeling(single_article_df, evaluation_size = 6) #Need to applied in component 2 and only train, test dfs need to be there
        train_df1=train_df1.append(train_df)
        eval_df1=eval_df1.append(evaluation_df)
    #train_df1.to_csv(train_df_path.path,index = False)
    #eval_df1.to_csv(validation_df_path.path,index = False)


In [ ]:
train_df1


In [ ]:
for article in monthly_aggrigated_ecommerce_data[ID_COL].unique():
    single_article_training_df =train_df1[train_df1[ID_COL] == article]
    single_article_evaluation_df = eval_df1[eval_df1[ID_COL] == article]
   
    article_df=single_article_training_df.append(single_article_evaluation_df,ignore_index=True)


In [ ]:
article_df

In [ ]:

from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import datetime
def ceil_list_of_values(values):
    return np.ceil(values)

def generate_dates(last_date, horizon):
    print("generate dates started!!!")
    dates_list = list(pd.date_range(start=last_date , periods=horizon + 1,freq="M"))
    print("dates SUCESSFUL!!!")
    return dates_list[1:]

def generate_algorithm_results_df(eval_df, model_predictions, model_forecasts, algorithm_name):
        FORECAST_HORIZON=18
        print("algorithem started!!!")
        eval_df[TARGET_COL+"_PREDICTED"] = model_predictions
        last_date = eval_df[DATE_COL].values.tolist()[-1] #last date
        projected_dates = generate_dates(last_date, FORECAST_HORIZON)
        forecast_df = pd.DataFrame()
        forecast_df[DATE_COL] = projected_dates
        forecast_df[ID_COL] = eval_df[ID_COL].values[0]
        #forecast_df["CUSTOMER_NAME"] = eval_df["CUSTOMER_NAME_"].values[0] #comment for primary and secondary retail
        forecast_df[TARGET_COL+"_Forecasted"] = np.ceil(model_forecasts)
        final_outcome_df = pd.concat([eval_df, forecast_df], axis=0)
        final_outcome_df["MODEL"] = algorithm_name
        print("algorithem SUCESSFUL!!!")
        return final_outcome_df

def ma_train_with_optimal_parameters(best_window, article_df):
        series = article_df[TARGET_COL].values.tolist()
        preds = [] #to append predictions
        for t in range(FORECAST_HORIZON):
            length = len(series)
            forecast = np.mean([series[i] for i in range(length - best_window, length)])

            #append the forecast to the history and predictions
            series.append(forecast)
            preds.append(forecast)   
        return preds

def MAMODEL(train, evaluation, article_df):
    best_rmse = float('inf')
    best_window = None
    evaluation_predictions = None
    test_data = None
    data_length = len(article_df[TARGET_COL].values.tolist())
    windows = [x for x in range(2, data_length // 2)]
    

    #iterate through windows
    for window in windows:
        #history and test series
        series = article_df[TARGET_COL].values.tolist()
        history = [series[i] for i in range(window)]
        test = [series[i] for i in range(window, len(series))]

        predictions = [] #predictions

        #walk forward over time steps in test data
        for t in range(len(test)):
            length = len(history)
            yhat = np.mean([history[i] for i in range(length-window, length)])
            obs = test[t]

            predictions.append(yhat)
            history.append(obs)

        #error rate
        error_rate = np.sqrt(mean_squared_error(test, predictions))

        #best parameters
        if best_rmse >= error_rate:
            best_rmse = error_rate
            evaluation_predictions = predictions
            best_window = window
            test_data = test

    forecast_predictions =  ma_train_with_optimal_parameters(best_window, article_df)   
    evaluation_predictions = ceil_list_of_values(evaluation_predictions[-len(evaluation):])
    forecast_predictions = ceil_list_of_values(forecast_predictions)
    return evaluation_predictions, forecast_predictions

algorithem_df=pd.DataFrame()
for article in monthly_aggrigated_ecommerce_data[ID_COL].unique():
    print(article)
    single_article_training_df =train_df1[train_df1[ID_COL] == article]
    single_article_evaluation_df = eval_df1[eval_df1[ID_COL] == article]
    try:
        
        article_df=single_article_training_df.append(single_article_evaluation_df,ignore_index=True)

        model_evaluation_predictions, model_forecast_predictions =  MAMODEL(single_article_training_df, single_article_evaluation_df, article_df)
        print("fg")
        algorithm_results = generate_algorithm_results_df(single_article_evaluation_df, model_evaluation_predictions, model_forecast_predictions, "MA")
        algorithem_df=algorithem_df.append(algorithm_results,ignore_index=True)
        print(algorithm_results)
        print(algorithem_df.head())
        
        
    except Exception as e:
        print(e)
        continue
algorithem_df.to_csv("algo_df.csv",index = False)




In [ ]:
fg=pd.read_csv("algo_df.csv")

In [ ]:
fg